In [2]:
!pip install -U pip wheel setuptools
!pip install concrete-python

  Using cached concrete_python-2.9.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (12 kB)
  Using cached z3_solver-4.13.0.0-py2.py3-none-manylinux2014_x86_64.whl.metadata (757 bytes)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.

# Workflow de Concrete


In [4]:
from concrete import fhe
# Definir la funcion
def multiply(x, y):
    return x + y

# Definir el compilador
compiler = fhe.Compiler(multiply, {"x": "encrypted", "y": "encrypted"})

# Definir el conjunto de entrada(determina la distribucion)
inputset = [(2, 3), (0, 0), (1, 6), (7, 7), (7, 1), (3, 2), (6, 1), (1, 7), (4, 5), (5, 4)]

# Compilar en el circuito
print(f"Compilation...")
circuit = compiler.compile(inputset)

# Generar llave del circuito
print(f"Key generation...")
circuit.keygen()

# Encriptar(encrypt), realizar la operacion(run) y desencriptar(decrypt)
print(f"Homomorphic evaluation...")
encrypted_x, encrypted_y = circuit.encrypt(2, 6)
encrypted_result = circuit.run(encrypted_x, encrypted_y)
result = circuit.decrypt(encrypted_result)

assert result == multiply(2, 6)

Compilation...
Key generation...
Homomorphic evaluation...


## Minimos cuadrados con las uñas

In [5]:
import numpy as np

# Datos de ejemplo (X con dos características + columna de unos)
X = np.array([[1, 2],
              [2, 3],
              [3, 5],
              [4, 7],
              [5, 11]])

y = np.array([2.2, 2.8, 3.6, 4.5, 5.1])  # Vector de salida

# Agregar columna de unos para el término independiente
X = np.hstack([X, np.ones((X.shape[0], 1))])

# Calcular los coeficientes beta usando la ecuación normal
beta = np.linalg.inv(X.T @ X) @ X.T @ y

print(f"Coeficientes: {beta}")

Coeficientes: [ 0.76571429 -0.00714286  1.38285714]


## Intento fallido de usar descenso del gradiente

In [8]:
import numpy as np
from concrete import fhe

# Definir la función de actualización de parámetros para el descenso de gradiente
def gradient_step(X, y, beta, alpha):
    """
    Un solo paso de descenso de gradiente para la regresión de mínimos cuadrados.

    X: Matriz de características
    y: Vector de salida
    beta: Coeficientes actuales
    alpha: Tasa de aprendizaje (pequeña)

    Retorna nuevos coeficientes beta después de una actualización.
    """
    # Predicción
    y_pred = np.dot(X, beta)

    # Cálculo del gradiente
    error = y_pred - y
    gradient = (X.T @ error) / len(y)

    # Actualización de los coeficientes
    beta_new = beta - alpha * gradient

    return beta_new

# Definir la función para el circuito FHE
def encrypted_gradient_step(X, y, beta, alpha):
    """
    Ejecuta un paso de descenso de gradiente sobre datos encriptados.
    """
    return gradient_step(X, y, beta, alpha)

# Crear datos de ejemplo
X = np.array([[1, 2], [2, 3], [3, 5], [4, 7], [5, 11]], dtype=np.int64)
y = np.array([2, 3, 5, 7, 11], dtype=np.int64)

# Agregar columna de unos para el término independiente
X = np.hstack([X, np.ones((X.shape[0], 1), dtype=np.int64)])

# Inicializar beta (coeficientes) en ceros
beta = np.zeros(X.shape[1], dtype=np.int64)

# Tasa de aprendizaje pequeña
alpha = 1  # Tiene que ser entero en FHE

# Compilación del circuito FHE
compiler = fhe.Compiler(encrypted_gradient_step, {"X": "encrypted", "y": "encrypted", "beta": "encrypted", "alpha": "encrypted"})

# Definir el conjunto de entrada (necesario para FHE)
inputset = [(X, y, beta, alpha)]

print("Compiling...")
circuit = compiler.compile(inputset)

# Generar llave del circuito
print("Key generation...")
circuit.keygen()

# Encriptar los datos
encrypted_X = circuit.encrypt(X)
encrypted_y = circuit.encrypt(y)
encrypted_beta = circuit.encrypt(beta)
encrypted_alpha = circuit.encrypt(alpha)

# Ejecutar el circuito sobre datos encriptados
print("Running encrypted gradient step...")
encrypted_beta_new = circuit.run(encrypted_X, encrypted_y, encrypted_beta, encrypted_alpha)

# Desencriptar el resultado
beta_new = circuit.decrypt(encrypted_beta_new)

print(f"Coeficientes estimados después de un paso: {beta_new}")


Compiling...


RuntimeError: Function you are trying to compile cannot be compiled

 %0 = X                        # EncryptedTensor<uint4, shape=(5, 3)>        ∈ [1, 11]
 %1 = y                        # EncryptedTensor<uint4, shape=(5,)>          ∈ [2, 11]
 %2 = beta                     # EncryptedTensor<uint1, shape=(3,)>          ∈ [0, 0]
 %3 = alpha                    # EncryptedScalar<uint1>                      ∈ [1, 1]
 %4 = dot(%0, %2)              # EncryptedTensor<uint1, shape=(5,)>          ∈ [0, 0]
 %5 = subtract(%4, %1)         # EncryptedTensor<int5, shape=(5,)>           ∈ [-11, -2]
 %6 = transpose(%0)            # EncryptedTensor<uint4, shape=(3, 5)>        ∈ [1, 11]
 %7 = matmul(%6, %5)           # EncryptedTensor<int9, shape=(3,)>           ∈ [-208, -28]
 %8 = 5                        # ClearScalar<uint3>                          ∈ [5, 5]
 %9 = divide(%7, %8)           # EncryptedTensor<float64, shape=(3,)>        ∈ [-41.6, -5.6]
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [10]:
!pip install concrete-python[full]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pygraphviz
Failed to build pygraphviz
ERROR: Failed to build installable wheels for some pyproject.toml based projects (pygraphviz)


In [9]:
import numpy as np
from concrete import fhe
from concrete.fhe import FHElinalg

# Función para resolver el sistema de ecuaciones normales
def solve_least_squares(X, y):
    """
    Calcula los coeficientes de mínimos cuadrados resolviendo (X^T X) beta = X^T y
    """
    XT = X.T  # Transpuesta de X
    XTX = FHElinalg.matmul(XT, X)  # Multiplicación de matrices en FHE
    XTy = FHElinalg.matmul(XT, y)  # Producto X^T y

    # Resolver el sistema XTX * beta = XTy usando FHElinalg.solve
    beta = FHElinalg.solve(XTX, XTy)

    return beta

# Datos de entrada (Matriz X y vector y)
X = np.array([[1, 2], [2, 3], [3, 5], [4, 7], [5, 11]], dtype=np.int64)
y = np.array([2, 3, 5, 7, 11], dtype=np.int64)

# Agregar una columna de unos para el término independiente
X = np.hstack([X, np.ones((X.shape[0], 1), dtype=np.int64)])

# Compilar la función en FHE
compiler = fhe.Compiler(solve_least_squares, {"X": "encrypted", "y": "encrypted"})

# Definir el conjunto de entrada para la compilación
inputset = [(X, y)]

print("Compiling...")
circuit = compiler.compile(inputset)

# Generar llaves FHE
print("Key generation...")
circuit.keygen()

# Encriptar los datos
encrypted_X = circuit.encrypt(X)
encrypted_y = circuit.encrypt(y)

# Ejecutar el circuito sobre datos encriptados
print("Running encrypted least squares...")
encrypted_beta = circuit.run(encrypted_X, encrypted_y)

# Desencriptar el resultado
beta = circuit.decrypt(encrypted_beta)

print(f"Coeficientes estimados: {beta}")


ImportError: cannot import name 'FHElinalg' from 'concrete.fhe' (/usr/local/lib/python3.11/dist-packages/concrete/fhe/__init__.py)